In [1]:
import nibabel as nib
import matplotlib.pyplot as plt
from scipy import ndimage
import cv2
import glob
import nilearn
from nilearn import image
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib

In [2]:
train1=pd.read_csv('t1w_train_fold1.csv', index_col=0)
train1.groupby('Label').count()

,T1w_file,day
Label,,
AD,20,20
MCI,10,10
Normal,115,115


In [12]:
train2=pd.read_csv('t1w_train_fold2.csv', index_col=0)
train2

,T1w_file,day,Label
Subject,,,
OAS30064,rsub-OAS30064_ses-d0687_run-01_T1w.nii,687,AD
OAS30094,rsub-OAS30094_ses-d0679_run-01_T1w.nii,679,AD
OAS30119,rsub-OAS30119_ses-d1209_run-01_T1w.nii,1209,AD
OAS30144,rsub-OAS30144_ses-d1204_run-01_T1w.nii,1204,AD
OAS30145,rsub-OAS30145_ses-d3506_run-01_T1w.nii,3506,AD
...,...,...,...
OAS30393,rsub-OAS30393_ses-d6201_run-01_T1w.nii,6201,Normal
OAS30393,rsub-OAS30393_ses-d6803_run-01_T1w.nii,6803,Normal
OAS30402,rsub-OAS30402_ses-d1085_T1w.nii,1085,Normal


In [ ]:
train_labels=[]
train_file_names=[]
dftrain=train1
for i in range(len(dftrain['T1w_file'])):
    file_name=dftrain.iloc[i].T1w_file
    label=dftrain.iloc[i].Label

    mri = nib.load('/raid/aidana_massalimova/nii_data/T1w/'+file_name)
    zooms = mri.header.get_zooms()
    old_affine = mri.affine
    new_affine = nib.affines.rescale_affine(affine=old_affine, shape=mri.get_fdata().shape, zooms=zooms, new_shape=(256,256,256))
    new_mri = image.resample_img(mri, target_affine=new_affine,target_shape=(256,256,256), interpolation='nearest')    
    epi_img_data = new_mri.get_fdata()
    if label=='Normal':
        slice_0 = epi_img_data[256//2, :, :]
        slice_1 = epi_img_data[:, 256//2, :]
        slice_2 = epi_img_data[:, :, 256//2]

        slice_0 /= slice_0.max()/255.0
        slice_1 /= slice_1.max()/255.0
        slice_2 /= slice_2.max()/255.0

        fft_p0 = slice_0.astype(np.uint8)
        fft_p1 = slice_1.astype(np.uint8)
        fft_p2 = slice_2.astype(np.uint8)
        mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
        mri = Image.fromarray(mri)
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
        mri.save('fold1/train/{}'.format(name))                        
    elif label=='MCI':
        for i in range(123,134):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)
            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold1/train/{}'.format(name))        
 

    elif label=='AD':
        for i in range(125,131):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)

            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold1/train/{}'.format(name))    
            

In [4]:
train_labels=[]
train_file_names=[]
dftrain=train1
for i in range(len(dftrain['T1w_file'])):
    file_name=dftrain.iloc[i].T1w_file
    label=dftrain.iloc[i].Label
    if label=='Normal':
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
    elif label=='MCI':
        for i in range(123,134):
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
    elif label=='AD':
        for i in range(125,131):
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
fold1_train={'File_name':train_file_names,
      'Label':train_labels}  
df_fold1_train= pd.DataFrame (fold1_train, columns =  ['File_name','Label'])
df_fold1_train
from sklearn.utils import shuffle
df_fold1_train = shuffle(df_fold1_train)
np.savetxt('train_fold1.txt', df_fold1_train.values,delimiter=" ", fmt="%s")
df_fold1_train.groupby('Label').count()

,File_name
Label,
AD,120
MCI,110
Normal,115


In [6]:
train_labels=[]
train_file_names=[]
dftrain=train2
for i in range(len(dftrain['T1w_file'])):
    file_name=dftrain.iloc[i].T1w_file
    label=dftrain.iloc[i].Label

    mri = nib.load('/raid/aidana_massalimova/nii_data/T1w/'+file_name)
    zooms = mri.header.get_zooms()
    old_affine = mri.affine
    new_affine = nib.affines.rescale_affine(affine=old_affine, shape=mri.get_fdata().shape, zooms=zooms, new_shape=(256,256,256))
    new_mri = image.resample_img(mri, target_affine=new_affine,target_shape=(256,256,256), interpolation='nearest')    
    epi_img_data = new_mri.get_fdata()
    if label=='Normal':
        slice_0 = epi_img_data[256//2, :, :]
        slice_1 = epi_img_data[:, 256//2, :]
        slice_2 = epi_img_data[:, :, 256//2]

        slice_0 /= slice_0.max()/255.0
        slice_1 /= slice_1.max()/255.0
        slice_2 /= slice_2.max()/255.0

        fft_p0 = slice_0.astype(np.uint8)
        fft_p1 = slice_1.astype(np.uint8)
        fft_p2 = slice_2.astype(np.uint8)
        mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
        mri = Image.fromarray(mri)
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
        mri.save('fold2/train/{}'.format(name))                        
    elif label=='MCI':
        for i in range(123,134):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)
            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold2/train/{}'.format(name))        
 

    elif label=='AD':
        for i in range(125,131):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)

            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold2/train/{}'.format(name))    
            


In [13]:
train_labels=[]
train_file_names=[]
dftrain=train2
for i in range(len(dftrain['T1w_file'])):
    file_name=dftrain.iloc[i].T1w_file
    label=dftrain.iloc[i].Label
    if label=='Normal':
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
    elif label=='MCI':
        for i in range(123,134):
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
    elif label=='AD':
        for i in range(125,131):
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
fold1_train={'File_name':train_file_names,
      'Label':train_labels}  
df_fold1_train= pd.DataFrame (fold1_train, columns =  ['File_name','Label'])
df_fold1_train.groupby('Label').count()
df_fold1_train
from sklearn.utils import shuffle
df_fold1_train = shuffle(df_fold1_train)
np.savetxt('train_fold2.txt', df_fold1_train.values,delimiter=" ", fmt="%s")
df_fold1_train.groupby('Label').count()


,File_name
Label,
AD,120
MCI,110
Normal,115


In [14]:
train3=pd.read_csv('t1w_train_fold3.csv', index_col=0)
train3

,T1w_file,day,Label
Subject,,,
OAS30064,rsub-OAS30064_ses-d0687_run-01_T1w.nii,687,AD
OAS30094,rsub-OAS30094_ses-d0679_run-01_T1w.nii,679,AD
OAS30119,rsub-OAS30119_ses-d1209_run-01_T1w.nii,1209,AD
OAS30144,rsub-OAS30144_ses-d1204_run-01_T1w.nii,1204,AD
OAS30145,rsub-OAS30145_ses-d3506_run-01_T1w.nii,3506,AD
...,...,...,...
OAS30393,rsub-OAS30393_ses-d6201_run-01_T1w.nii,6201,Normal
OAS30393,rsub-OAS30393_ses-d6803_run-01_T1w.nii,6803,Normal
OAS30402,rsub-OAS30402_ses-d1085_T1w.nii,1085,Normal


In [9]:
train_labels=[]
train_file_names=[]
dftrain=train3
for i in range(len(dftrain['T1w_file'])):
    file_name=dftrain.iloc[i].T1w_file
    label=dftrain.iloc[i].Label

    mri = nib.load('/raid/aidana_massalimova/nii_data/T1w/'+file_name)
    zooms = mri.header.get_zooms()
    old_affine = mri.affine
    new_affine = nib.affines.rescale_affine(affine=old_affine, shape=mri.get_fdata().shape, zooms=zooms, new_shape=(256,256,256))
    new_mri = image.resample_img(mri, target_affine=new_affine,target_shape=(256,256,256), interpolation='nearest')    
    epi_img_data = new_mri.get_fdata()
    if label=='Normal':
        slice_0 = epi_img_data[256//2, :, :]
        slice_1 = epi_img_data[:, 256//2, :]
        slice_2 = epi_img_data[:, :, 256//2]

        slice_0 /= slice_0.max()/255.0
        slice_1 /= slice_1.max()/255.0
        slice_2 /= slice_2.max()/255.0

        fft_p0 = slice_0.astype(np.uint8)
        fft_p1 = slice_1.astype(np.uint8)
        fft_p2 = slice_2.astype(np.uint8)
        mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
        mri = Image.fromarray(mri)
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
        mri.save('fold3/train/{}'.format(name))                        
    elif label=='MCI':
        for i in range(123,134):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)
            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold3/train/{}'.format(name))        
 

    elif label=='AD':
        for i in range(125,131):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)

            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold3/train/{}'.format(name))    
            


In [15]:
train_labels=[]
train_file_names=[]
dftrain=train3
for i in range(len(dftrain['T1w_file'])):
    file_name=dftrain.iloc[i].T1w_file
    label=dftrain.iloc[i].Label
    if label=='Normal':
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
    elif label=='MCI':
        for i in range(123,134):
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
    elif label=='AD':
        for i in range(125,131):
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
fold1_train={'File_name':train_file_names,
      'Label':train_labels}  
df_fold1_train= pd.DataFrame (fold1_train, columns =  ['File_name','Label'])
df_fold1_train.groupby('Label').count()
df_fold1_train
from sklearn.utils import shuffle
df_fold1_train = shuffle(df_fold1_train)
np.savetxt('train_fold3.txt', df_fold1_train.values,delimiter=" ", fmt="%s")
df_fold1_train.groupby('Label').count()


,File_name
Label,
AD,120
MCI,110
Normal,116


In [18]:
train4=pd.read_csv('t1w_train_fold4.csv', index_col=0)
train4.groupby('Label').count()

,T1w_file,day
Label,,
AD,20,20
MCI,11,11
Normal,115,115


In [12]:
train_labels=[]
train_file_names=[]
dftrain=train4
for i in range(len(dftrain['T1w_file'])):
    file_name=dftrain.iloc[i].T1w_file
    label=dftrain.iloc[i].Label

    mri = nib.load('/raid/aidana_massalimova/nii_data/T1w/'+file_name)
    zooms = mri.header.get_zooms()
    old_affine = mri.affine
    new_affine = nib.affines.rescale_affine(affine=old_affine, shape=mri.get_fdata().shape, zooms=zooms, new_shape=(256,256,256))
    new_mri = image.resample_img(mri, target_affine=new_affine,target_shape=(256,256,256), interpolation='nearest')    
    epi_img_data = new_mri.get_fdata()
    if label=='Normal':
        slice_0 = epi_img_data[256//2, :, :]
        slice_1 = epi_img_data[:, 256//2, :]
        slice_2 = epi_img_data[:, :, 256//2]

        slice_0 /= slice_0.max()/255.0
        slice_1 /= slice_1.max()/255.0
        slice_2 /= slice_2.max()/255.0

        fft_p0 = slice_0.astype(np.uint8)
        fft_p1 = slice_1.astype(np.uint8)
        fft_p2 = slice_2.astype(np.uint8)
        mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
        mri = Image.fromarray(mri)
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
        mri.save('fold4/train/{}'.format(name))                        
    elif label=='MCI':
        for i in range(123,134):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)
            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold4/train/{}'.format(name))        
 

    elif label=='AD':
        for i in range(125,131):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)

            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold4/train/{}'.format(name))    
            


In [17]:
train_labels=[]
train_file_names=[]
dftrain=train4
for i in range(len(dftrain['T1w_file'])):
    file_name=dftrain.iloc[i].T1w_file
    label=dftrain.iloc[i].Label
    if label=='Normal':
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
    elif label=='MCI':
        for i in range(123,134):
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
    elif label=='AD':
        for i in range(125,131):
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
fold1_train={'File_name':train_file_names,
      'Label':train_labels}  
df_fold1_train= pd.DataFrame (fold1_train, columns =  ['File_name','Label'])
df_fold1_train.groupby('Label').count()
df_fold1_train
from sklearn.utils import shuffle
df_fold1_train = shuffle(df_fold1_train)
np.savetxt('train_fold4.txt', df_fold1_train.values,delimiter=" ", fmt="%s")
df_fold1_train.groupby('Label').count()


,File_name
Label,
AD,120
MCI,121
Normal,115


In [20]:
train5=pd.read_csv('t1w_train_fold5.csv', index_col=0)
train5.groupby('Label').count()

,T1w_file,day
Label,,
AD,20,20
MCI,11,11
Normal,115,115


In [15]:
train_labels=[]
train_file_names=[]
dftrain=train5
for i in range(len(dftrain['T1w_file'])):
    file_name=dftrain.iloc[i].T1w_file
    label=dftrain.iloc[i].Label

    mri = nib.load('/raid/aidana_massalimova/nii_data/T1w/'+file_name)
    zooms = mri.header.get_zooms()
    old_affine = mri.affine
    new_affine = nib.affines.rescale_affine(affine=old_affine, shape=mri.get_fdata().shape, zooms=zooms, new_shape=(256,256,256))
    new_mri = image.resample_img(mri, target_affine=new_affine,target_shape=(256,256,256), interpolation='nearest')    
    epi_img_data = new_mri.get_fdata()
    if label=='Normal':
        slice_0 = epi_img_data[256//2, :, :]
        slice_1 = epi_img_data[:, 256//2, :]
        slice_2 = epi_img_data[:, :, 256//2]

        slice_0 /= slice_0.max()/255.0
        slice_1 /= slice_1.max()/255.0
        slice_2 /= slice_2.max()/255.0

        fft_p0 = slice_0.astype(np.uint8)
        fft_p1 = slice_1.astype(np.uint8)
        fft_p2 = slice_2.astype(np.uint8)
        mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
        mri = Image.fromarray(mri)
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
        mri.save('fold5/train/{}'.format(name))                        
    elif label=='MCI':
        for i in range(123,134):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)
            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold5/train/{}'.format(name))        
 

    elif label=='AD':
        for i in range(125,131):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)

            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold5/train/{}'.format(name))    
            


In [21]:
train_labels=[]
train_file_names=[]
dftrain=train5
for i in range(len(dftrain['T1w_file'])):
    file_name=dftrain.iloc[i].T1w_file
    label=dftrain.iloc[i].Label
    if label=='Normal':
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
    elif label=='MCI':
        for i in range(123,134):
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
    elif label=='AD':
        for i in range(125,131):
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
fold1_train={'File_name':train_file_names,
      'Label':train_labels}  
df_fold1_train= pd.DataFrame (fold1_train, columns =  ['File_name','Label'])
df_fold1_train.groupby('Label').count()
df_fold1_train
from sklearn.utils import shuffle
df_fold1_train = shuffle(df_fold1_train)
np.savetxt('train_fold5.txt', df_fold1_train.values,delimiter=" ", fmt="%s")
df_fold1_train.groupby('Label').count()


,File_name
Label,
AD,120
MCI,121
Normal,115


In [22]:
test1=pd.read_csv('t1w_test_fold1.csv', index_col=0)
test1

,T1w_file,day,Label
Subject,,,
OAS30064,rsub-OAS30064_ses-d0687_run-01_T1w.nii,687,AD
OAS30094,rsub-OAS30094_ses-d0679_run-01_T1w.nii,679,AD
OAS30119,rsub-OAS30119_ses-d1209_run-01_T1w.nii,1209,AD
OAS30144,rsub-OAS30144_ses-d1204_run-01_T1w.nii,1204,AD
OAS30145,rsub-OAS30145_ses-d3506_run-01_T1w.nii,3506,AD
OAS30012,rsub-OAS30012_ses-d0124_run-01_T1w.nii,124,MCI
OAS30012,rsub-OAS30012_ses-d0581_run-01_T1w.nii,581,MCI
OAS30041,rsub-OAS30041_ses-d6548_run-01_T1w.nii,6548,MCI
OAS30240,rsub-OAS30240_ses-d3843_T1w.nii,3843,Normal


In [18]:
train_labels=[]
train_file_names=[]
dftrain=test1
for i in range(len(dftrain['T1w_file'])):
    file_name=dftrain.iloc[i].T1w_file
    label=dftrain.iloc[i].Label

    mri = nib.load('/raid/aidana_massalimova/nii_data/T1w/'+file_name)
    zooms = mri.header.get_zooms()
    old_affine = mri.affine
    new_affine = nib.affines.rescale_affine(affine=old_affine, shape=mri.get_fdata().shape, zooms=zooms, new_shape=(256,256,256))
    new_mri = image.resample_img(mri, target_affine=new_affine,target_shape=(256,256,256), interpolation='nearest')    
    epi_img_data = new_mri.get_fdata()
    if label=='Normal':
        slice_0 = epi_img_data[256//2, :, :]
        slice_1 = epi_img_data[:, 256//2, :]
        slice_2 = epi_img_data[:, :, 256//2]

        slice_0 /= slice_0.max()/255.0
        slice_1 /= slice_1.max()/255.0
        slice_2 /= slice_2.max()/255.0

        fft_p0 = slice_0.astype(np.uint8)
        fft_p1 = slice_1.astype(np.uint8)
        fft_p2 = slice_2.astype(np.uint8)
        mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
        mri = Image.fromarray(mri)
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
        mri.save('fold1/test/{}'.format(name))                        
    elif label=='MCI':
        for i in range(124,138):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)
            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold1/test/{}'.format(name))        
 

    elif label=='AD':
        for i in range(125,131):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)

            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold1/test/{}'.format(name))    
            


In [24]:
train_labels=[]
train_file_names=[]
dftrain=test1
for i in range(len(dftrain['T1w_file'])):
    file_name=dftrain.iloc[i].T1w_file
    label=dftrain.iloc[i].Label
    if label=='Normal':
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
    elif label=='MCI':
        for i in range(124,134):
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
    elif label=='AD':
        for i in range(125,131):
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
fold1_train={'File_name':train_file_names,
      'Label':train_labels}  
df_fold1_train= pd.DataFrame (fold1_train, columns =  ['File_name','Label'])
df_fold1_train.groupby('Label').count()
df_fold1_train
from sklearn.utils import shuffle
df_fold1_train = shuffle(df_fold1_train)
np.savetxt('test_fold1.txt', df_fold1_train.values,delimiter=" ", fmt="%s")
df_fold1_train.groupby('Label').count()


,File_name
Label,
AD,30
MCI,30
Normal,29


In [25]:
test2=pd.read_csv('t1w_test_fold2.csv', index_col=0)


In [21]:
train_labels=[]
train_file_names=[]
dftrain=test2
for i in range(len(dftrain['T1w_file'])):
    file_name=dftrain.iloc[i].T1w_file
    label=dftrain.iloc[i].Label

    mri = nib.load('/raid/aidana_massalimova/nii_data/T1w/'+file_name)
    zooms = mri.header.get_zooms()
    old_affine = mri.affine
    new_affine = nib.affines.rescale_affine(affine=old_affine, shape=mri.get_fdata().shape, zooms=zooms, new_shape=(256,256,256))
    new_mri = image.resample_img(mri, target_affine=new_affine,target_shape=(256,256,256), interpolation='nearest')    
    epi_img_data = new_mri.get_fdata()
    if label=='Normal':
        slice_0 = epi_img_data[256//2, :, :]
        slice_1 = epi_img_data[:, 256//2, :]
        slice_2 = epi_img_data[:, :, 256//2]

        slice_0 /= slice_0.max()/255.0
        slice_1 /= slice_1.max()/255.0
        slice_2 /= slice_2.max()/255.0

        fft_p0 = slice_0.astype(np.uint8)
        fft_p1 = slice_1.astype(np.uint8)
        fft_p2 = slice_2.astype(np.uint8)
        mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
        mri = Image.fromarray(mri)
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
        mri.save('fold2/test/{}'.format(name))                        
    elif label=='MCI':
        for i in range(124,138):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)
            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold2/test/{}'.format(name))        
 

    elif label=='AD':
        for i in range(125,131):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)

            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold2/test/{}'.format(name))    
            


In [27]:
train_labels=[]
train_file_names=[]
dftrain=test2
for i in range(len(dftrain['T1w_file'])):
    file_name=dftrain.iloc[i].T1w_file
    label=dftrain.iloc[i].Label
    if label=='Normal':
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
    elif label=='MCI':
        for i in range(124,134):
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
    elif label=='AD':
        for i in range(125,131):
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
fold1_train={'File_name':train_file_names,
      'Label':train_labels}  
df_fold1_train= pd.DataFrame (fold1_train, columns =  ['File_name','Label'])
df_fold1_train.groupby('Label').count()
df_fold1_train
from sklearn.utils import shuffle
df_fold1_train = shuffle(df_fold1_train)
np.savetxt('test_fold2.txt', df_fold1_train.values,delimiter=" ", fmt="%s")
df_fold1_train.groupby('Label').count()


,File_name
Label,
AD,30
MCI,30
Normal,29


In [28]:
test3=pd.read_csv('t1w_test_fold3.csv', index_col=0)
test3.groupby('Label').count()

,T1w_file,day
Label,,
AD,5,5
MCI,3,3
Normal,28,28


In [24]:
train_labels=[]
train_file_names=[]
dftrain=test3
for i in range(len(dftrain['T1w_file'])):
    file_name=dftrain.iloc[i].T1w_file
    label=dftrain.iloc[i].Label

    mri = nib.load('/raid/aidana_massalimova/nii_data/T1w/'+file_name)
    zooms = mri.header.get_zooms()
    old_affine = mri.affine
    new_affine = nib.affines.rescale_affine(affine=old_affine, shape=mri.get_fdata().shape, zooms=zooms, new_shape=(256,256,256))
    new_mri = image.resample_img(mri, target_affine=new_affine,target_shape=(256,256,256), interpolation='nearest')    
    epi_img_data = new_mri.get_fdata()
    if label=='Normal':
        slice_0 = epi_img_data[256//2, :, :]
        slice_1 = epi_img_data[:, 256//2, :]
        slice_2 = epi_img_data[:, :, 256//2]

        slice_0 /= slice_0.max()/255.0
        slice_1 /= slice_1.max()/255.0
        slice_2 /= slice_2.max()/255.0

        fft_p0 = slice_0.astype(np.uint8)
        fft_p1 = slice_1.astype(np.uint8)
        fft_p2 = slice_2.astype(np.uint8)
        mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
        mri = Image.fromarray(mri)
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
        mri.save('fold3/test/{}'.format(name))                        
    elif label=='MCI':
        for i in range(124,131):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)
            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold3/test/{}'.format(name))        
 

    elif label=='AD':
        for i in range(125,131):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)

            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold3/test/{}'.format(name))    
            


In [29]:
train_labels=[]
train_file_names=[]
dftrain=test3
for i in range(len(dftrain['T1w_file'])):
    file_name=dftrain.iloc[i].T1w_file
    label=dftrain.iloc[i].Label
    if label=='Normal':
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
    elif label=='MCI':
        for i in range(124,134):
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
    elif label=='AD':
        for i in range(125,131):
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
fold1_train={'File_name':train_file_names,
      'Label':train_labels}  
df_fold1_train= pd.DataFrame (fold1_train, columns =  ['File_name','Label'])
df_fold1_train.groupby('Label').count()
df_fold1_train
from sklearn.utils import shuffle
df_fold1_train = shuffle(df_fold1_train)
np.savetxt('test_fold3.txt', df_fold1_train.values,delimiter=" ", fmt="%s")
df_fold1_train.groupby('Label').count()


,File_name
Label,
AD,30
MCI,30
Normal,28


In [30]:
test4=pd.read_csv('t1w_test_fold4.csv', index_col=0)


In [27]:
train_labels=[]
train_file_names=[]
dftrain=test4
for i in range(len(dftrain['T1w_file'])):
    file_name=dftrain.iloc[i].T1w_file
    label=dftrain.iloc[i].Label

    mri = nib.load('/raid/aidana_massalimova/nii_data/T1w/'+file_name)
    zooms = mri.header.get_zooms()
    old_affine = mri.affine
    new_affine = nib.affines.rescale_affine(affine=old_affine, shape=mri.get_fdata().shape, zooms=zooms, new_shape=(256,256,256))
    new_mri = image.resample_img(mri, target_affine=new_affine,target_shape=(256,256,256), interpolation='nearest')    
    epi_img_data = new_mri.get_fdata()
    if label=='Normal':
        slice_0 = epi_img_data[256//2, :, :]
        slice_1 = epi_img_data[:, 256//2, :]
        slice_2 = epi_img_data[:, :, 256//2]

        slice_0 /= slice_0.max()/255.0
        slice_1 /= slice_1.max()/255.0
        slice_2 /= slice_2.max()/255.0

        fft_p0 = slice_0.astype(np.uint8)
        fft_p1 = slice_1.astype(np.uint8)
        fft_p2 = slice_2.astype(np.uint8)
        mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
        mri = Image.fromarray(mri)
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
        mri.save('fold4/test/{}'.format(name))                        
    elif label=='MCI':
        for i in range(124,138):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)
            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold4/test/{}'.format(name))        
 

    elif label=='AD':
        for i in range(125,131):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)

            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold4/test/{}'.format(name))    
            


In [31]:
train_labels=[]
train_file_names=[]
dftrain=test4
for i in range(len(dftrain['T1w_file'])):
    file_name=dftrain.iloc[i].T1w_file
    label=dftrain.iloc[i].Label
    if label=='Normal':
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
    elif label=='MCI':
        for i in range(124,138):
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
    elif label=='AD':
        for i in range(125,131):
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
fold1_train={'File_name':train_file_names,
      'Label':train_labels}  
df_fold1_train= pd.DataFrame (fold1_train, columns =  ['File_name','Label'])
df_fold1_train.groupby('Label').count()
df_fold1_train
from sklearn.utils import shuffle
df_fold1_train = shuffle(df_fold1_train)
np.savetxt('test_fold4.txt', df_fold1_train.values,delimiter=" ", fmt="%s")
df_fold1_train.groupby('Label').count()
df_fold1_train.groupby('Label').count()


,File_name
Label,
AD,30
MCI,28
Normal,29


In [32]:
test5=pd.read_csv('t1w_test_fold5.csv', index_col=0)


In [ ]:
train_labels=[]
train_file_names=[]
dftrain=test5
for i in range(len(dftrain['T1w_file'])):
    file_name=dftrain.iloc[i].T1w_file
    label=dftrain.iloc[i].Label

    mri = nib.load('/raid/aidana_massalimova/nii_data/T1w/'+file_name)
    zooms = mri.header.get_zooms()
    old_affine = mri.affine
    new_affine = nib.affines.rescale_affine(affine=old_affine, shape=mri.get_fdata().shape, zooms=zooms, new_shape=(256,256,256))
    new_mri = image.resample_img(mri, target_affine=new_affine,target_shape=(256,256,256), interpolation='nearest')    
    epi_img_data = new_mri.get_fdata()
    if label=='Normal':
        slice_0 = epi_img_data[256//2, :, :]
        slice_1 = epi_img_data[:, 256//2, :]
        slice_2 = epi_img_data[:, :, 256//2]

        slice_0 /= slice_0.max()/255.0
        slice_1 /= slice_1.max()/255.0
        slice_2 /= slice_2.max()/255.0

        fft_p0 = slice_0.astype(np.uint8)
        fft_p1 = slice_1.astype(np.uint8)
        fft_p2 = slice_2.astype(np.uint8)
        mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
        mri = Image.fromarray(mri)
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
        mri.save('fold5/test/{}'.format(name))                        
    elif label=='MCI':
        for i in range(124,138):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)
            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold5/test/{}'.format(name))        
 

    elif label=='AD':
        for i in range(125,131):
            slice_0 = epi_img_data[i, :, :]
            slice_1 = epi_img_data[:, i, :]
            slice_2 = epi_img_data[:, :, i]

            slice_0 /= slice_0.max()/255.0
            slice_1 /= slice_1.max()/255.0
            slice_2 /= slice_2.max()/255.0

            fft_p0 = slice_0.astype(np.uint8)
            fft_p1 = slice_1.astype(np.uint8)
            fft_p2 = slice_2.astype(np.uint8)

            mri = np.stack([fft_p0, fft_p1, fft_p2], axis=2)  
            mri = Image.fromarray(mri)
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
            mri.save('fold5/test/{}'.format(name))    
            


In [ ]:
train_labels=[]
train_file_names=[]
dftrain=test5
for i in range(len(dftrain['T1w_file'])):
    file_name=dftrain.iloc[i].T1w_file
    label=dftrain.iloc[i].Label
    if label=='Normal':
        name=file_name[0:-4]+'_1.png'
        train_labels.append(label)
        train_file_names.append(name)
    elif label=='MCI':
        for i in range(124,138):
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
    elif label=='AD':
        for i in range(125,131):
            name=file_name[0:-4]+'_{}.png'.format(i)
            train_file_names.append(name)
            train_labels.append(label)
fold1_train={'File_name':train_file_names,
      'Label':train_labels}  
df_fold1_train= pd.DataFrame (fold1_train, columns =  ['File_name','Label'])
df_fold1_train.groupby('Label').count()
df_fold1_train
from sklearn.utils import shuffle
df_fold1_train = shuffle(df_fold1_train)
np.savetxt('test_fold5.txt', df_fold1_train.values,delimiter=" ", fmt="%s")
df_fold1_train.groupby('Label').count()
